In [7]:
import pandas as pd
import numpy as np
import time
from sklearn.impute import SimpleImputer

In [8]:
# variables
max_correlation = 0.7
min_correlation = 0.3
data_path = "../data/zoonosis_dataset_full.csv"

In [9]:
data = pd.read_csv(data_path)

In [10]:
data = data.loc[:, (data != 0).any(axis=0)]
corrmatrix = data.corr(numeric_only=True)
corrmatrix.head()

,TT_p2_Bias_HA,V_aa_Bias_HA,GGC_Bias_HA,AG_Bias_HA,CT_Bias_HA,ATA_Bias_HA,GG_p1_Bias_HA,G_Bias_HA,GA_p1_Bias_HA,A_Bias_HA,...,PAAC_Xc1.L_PB2,PAAC_Xc1.A_PB2,PAAC_Xc1.P_PB2,PAAC_Xc2.lambda3_PB2,PAAC_Xc1.F_PB2,PAAC_Xc1.C_PB2,PAAC_Xc1.I_PB2,PAAC_Xc1.W_PB2,PAAC_Xc1.R_PB2,PAAC_Xc1.H_PB2
TT_p2_Bias_HA,1.000000,0.033501,-0.020122,0.085947,-0.225044,-0.296177,-0.317045,0.130810,0.142579,-0.235783,...,-0.054508,0.013258,-0.054968,0.112929,-0.054959,-0.010397,-0.052938,-0.025831,0.076750,-0.043338
V_aa_Bias_HA,0.033501,1.000000,0.130224,-0.228245,-0.007940,-0.173006,-0.410107,-0.158081,0.017504,0.012478,...,0.032851,-0.020412,0.032951,-0.039131,0.032900,0.026322,0.034042,0.017771,0.000768,-0.045977
GGC_Bias_HA,-0.020122,0.130224,1.000000,-0.164258,0.157692,-0.441133,0.254086,0.117563,-0.315574,-0.443027,...,-0.094131,-0.074194,-0.093214,0.151112,-0.093448,-0.033935,-0.093476,-0.085762,0.073489,-0.141113
AG_Bias_HA,0.085947,-0.228245,-0.164258,1.000000,0.081191,0.071141,-0.272699,0.078902,0.185137,-0.175061,...,0.053349,0.074986,0.053335,-0.053274,0.053010,0.006943,0.052049,0.034633,0.029012,0.061516
CT_Bias_HA,-0.225044,-0.007940,0.157692,0.081191,1.000000,0.041057,0.133938,0.248622,-0.309992,-0.283248,...,-0.017463,-0.015114,-0.016644,0.004233,-0.017253,-0.018377,-0.017946,-0.030907,-0.025684,-0.070570


## Removing columns that are highly correlated

In [11]:
highlycorr = (corrmatrix > max_correlation).sum()
corr_counts = highlycorr.loc[highlycorr > 1]
highest_corr = highlycorr.idxmax()

In [12]:
while len(corr_counts) > 0:
    columns_to_drop = set(
        corrmatrix[highest_corr].loc[corrmatrix[highest_corr] > max_correlation].index
    )
    columns_to_drop.remove(highest_corr)
    data.drop(columns_to_drop, axis=1, inplace=True)
    corrmatrix = data.corr(numeric_only=True)
    highlycorr = (corrmatrix > max_correlation).sum()
    corr_counts = highlycorr.loc[highlycorr > 1]
    highest_corr = highlycorr.idxmax()
    print(len(corr_counts), data.shape)

225 (19531, 795)
102 (19531, 677)
91 (19531, 667)
86 (19531, 663)
81 (19531, 659)
77 (19531, 656)
73 (19531, 653)
69 (19531, 651)
66 (19531, 649)
63 (19531, 647)
60 (19531, 645)
57 (19531, 643)
54 (19531, 641)
50 (19531, 639)
48 (19531, 638)
46 (19531, 637)
44 (19531, 636)
42 (19531, 635)
40 (19531, 634)
38 (19531, 633)
36 (19531, 632)
34 (19531, 631)
32 (19531, 630)
30 (19531, 629)
28 (19531, 628)
26 (19531, 627)
24 (19531, 626)
22 (19531, 625)
20 (19531, 624)
18 (19531, 623)
16 (19531, 622)
14 (19531, 621)
12 (19531, 620)
10 (19531, 619)
8 (19531, 618)
6 (19531, 617)
4 (19531, 616)
2 (19531, 615)
0 (19531, 614)


In [13]:
data.to_csv(f"../cleaned_data/full_data_no_corrs_{max_correlation}.csv", index=False)

## Removing columns uncorrelated with output

In [14]:
data["label"] = data["label"].replace({"nz": 0, "hzoon": 1})
corrmatrix_low = data.corr(numeric_only=True)
columns_to_drop = set(
    corrmatrix_low["label"].loc[abs(corrmatrix_low["label"]) < min_correlation].index
)
data.drop(columns_to_drop, axis=1, inplace=True)

/var/folders/sx/g9qrr1y94z1gtsqjsbkjw2m40000gs/T/ipykernel_38357/4289826363.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data["label"] = data["label"].replace({"nz": 0, "hzoon": 1})


In [15]:
data.to_csv(
    f"../cleaned_data/full_data_no_corrs_{max_correlation}_label_corrs_{min_correlation}.csv",
    index=False,
)